In [1]:
import app
instance = app.create_app()
instance.app_context().push()

d:\dailydiet\dailydiet-api\venv\lib\site-packages\whitenoise\base.py:115: UserWarning: No directory at: D:\dailydiet\dailydiet-api\admin\static\
  warnings.warn(u"No directory at: {}".format(root))


In [2]:
from foods.models import Food,DietRecord
from extentions import elastic
from elasticsearch_dsl import Q,Search

In [ ]:
page = Food.query.paginate(page=1,per_page=100)

In [ ]:
page.next()

In [ ]:
page.pages

In [ ]:
page = Food.query.paginate(page=26,per_page=100)
while page.has_next:
    for item in page.items:
        Food.add_to_index(item)
    print(f'page {page.next_num - 1} indexed.')
    page = page.next()

In [ ]:
page = Food.query.paginate(page=51,per_page=100)

In [ ]:
#inserting last page
page = Food.query.paginate(page=60,per_page=100)
for item in page.items:
    Food.add_to_index(item)
print(f'page {page.pages} indexed.')

In [ ]:
query = "Mangos" 
#elastic ingredient search
search = elastic.search(
            index='ingredients',
            body={'query': {'multi_match': {'query': query, 'fields': ['food_name']}},
                  'from': 0, 'size': 20})


In [ ]:
search

In [7]:
q = Q('bool',
#          should=[
#             {
#               "multi_match": {
#                   "query": query,
#                   "fields": [
#                       "name^6.0",
#                       "category^1.0",
#                       "description^3.0",
#                       "tag_cloud^3.0",
#                       "ingredients^2.0",
#                       "directions^1.5",
#                       "author^1.0"
#                 ],
#                 "type": "phrase_prefix",
#                 "lenient": "true"
#               }
#             },
             
#          ],
      must=[
             {
                 "range":{
                     "nutrition.calories":{
                         "gte":300
                     }
                 }
             }
          ,
            {
                "match":{
                    "category":"sandwich"
                }
            }
         ],
#       boost=1,
#       minimum_should_match=1
     )

      
# category [one of 13 categories]
# calories [min:max]
# carbs [min:max]
# fats [min:max]
# proteins (per serving) [min:max]
# cook_time [min:max]
# prep_time [min:max]
# total_time [min:max]
# ingredients [list of ids]

In [10]:
s = Search(using=elastic)
s = s.query(q)
# for idd in [4914 ,2057,2042]:
#     s = s.filter('term',**{
#                "ingredient_ids": idd
#             })
s[0:2].execute().to_dict()

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 16, 'successful': 16, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 107, 'relation': 'eq'},
  'max_score': 4.080478,
  'hits': [{'_index': 'foods_new',
    '_type': '_doc',
    '_id': '927307',
    '_score': 4.080478,
    '_source': {'author': 'Ken Adams',
     'name': 'English Muffin Ham Breakfast Sandwich',
     'description': '',
     'category': 'sandwich',
     'nutrition': {'calories': 513.17,
      'carbs': 30.88,
      'fats': 27.58,
      'proteins': 36.13},
     'tag_cloud': 'English muffins Pepper Cheddar cheese Dairy Products Sliced ham Egg Sausages and Luncheon Meats Spices and Herbs realegg "English Muffin Ham Breakfast Sandwich" breakfast Salt Baked Products gluten',
     'ingredients': ['Egg',
      'Salt',
      'Pepper',
      'English muffins',
      'Sliced ham',
      'Cheddar cheese'],
     'ingredient_ids': [103, 221, 205, 4185, 905, 9],
     'directions': ['Preheat oven to 350 degrees F. Spray a j

In [ ]:
DietRecord.query.filter(DietRecord.ownerId == 1).order_by(DietRecord.generatedAt.desc()) \
        .limit(10).all()

In [ ]:
query= 'roast beef'
page=1
per_page=5
search = Search(using=elastic, index='foods_new')
elastic_query = Q('bool',
                should=[
                              {
                                  "multi_match": {
                                      "query": query,
                                      "fields": [
                                          "name^6.0",
                                          "category^1.0",
                                          "description^3.0",
                                          "tag_cloud^3.0",
                                          "ingredients^2.0",
                                          "directions^1.5",
                                          "author^1.0"
                                      ],
                                      "type": "phrase_prefix",
                                      "lenient": "true"
                                  }
                              },

                          ],
                          boost=1,
                          minimum_should_match=1)
from_index = (page - 1) * per_page
size = per_page
search = search.query(elastic_query)
search_results = search[from_index: from_index + size].execute()


In [ ]:
search_results.to_dict()